# P3 


This project gives you experience with getting data from the web. 



### Autograder Setup

The next code cell should be uncommented to run the autograder tests when using Colab/DeepNote. If you are using an environment with `otter-grader` already installed (your own machine, lab machines), then do not uncomment the code.

In [ ]:
# !pip install otter-grader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 168 kB 27.2 MB/s 
     |████████████████████████████████| 97 kB 6.6 MB/s 
     |████████████████████████████████| 297 kB 64.6 MB/s 
     |████████████████████████████████| 84 kB 2.9 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 


You will have access to a few tests for the project. Note, when you submit the autograder will run additional "hidden" tests on your solutions.

Please comment out this cell before submitting.

In [ ]:
# !wget https://pages.mtu.edu/~lebrown/un5550-f22/projects/p3/p3.files.zip
# !unzip p3.files.zip

--2022-10-17 02:06:59--  https://pages.mtu.edu/~lebrown/un5550-f22/projects/p3/p3.files.zip
Resolving pages.mtu.edu (pages.mtu.edu)... 141.219.70.232
Connecting to pages.mtu.edu (pages.mtu.edu)|141.219.70.232|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1730 (1.7K) [application/zip]
Saving to: ‘p3.files.zip’

p3.files.zip        100%[===================>]   1.69K  --.-KB/s    in 0s      

2022-10-17 02:07:00 (133 MB/s) - ‘p3.files.zip’ saved [1730/1730]

Archive:  p3.files.zip
   creating: tests/
  inflating: tests/q1.py             
  inflating: tests/q4.py             
  inflating: tests/q3.py             
  inflating: tests/q2.py             


In [ ]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import requests 
import re 
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns

import otter
grader = otter.Notebook()

**NOTE** A note on grading for this project.  
Each question has the totoal points listed which will be split between autograding methods and manual grading review your code to achieve the requested result.   

## Q1 - (5 pts total)

Scrape [https://www.data.gov](https://www.data.gov) to identify and print out the number of data sets available.

In [ ]:
site = requests.get('https://www.data.gov').text
q1 = BeautifulSoup(site, 'html5lib') 

In [ ]:
a1 = q1.find('div', class_ = "text-center getstarted").a.text 
total_datasets1 = re.sub('[^0-9]',"",a1.replace(",",""))    # total_datasets should be an integer  
total_datasets = int(total_datasets1[-6:]) 
# total_datasets    
print("The number of datasets is ", str(total_datasets)) 

The number of datasets is  335221


In [ ]:
grader.check("q1") 

q1 results: All test cases passed!

## Q2 - (20 pts total) 

You will be looking at the votes for U.S. Senate in the 117th Congress 1st session (2021).  In particular, we are seeking to identify the items that were rejected by a margin of less than 5 votes.  

Report in a DataFrame which items match this criteria.  
The DataFrame should have separate columns for the vote number, the tally, questions: description, and issue. 

Data available at: https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_117_1.htm

Consider completing one of the following methods: 
* using pandas read_html 
* using BeautifulSoup to parse the information. 

In [ ]:
url = "https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_117_1.htm" 
headers = {"User-agent": 
           "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.80 Safari/537.36"} 
site = requests.get(url, headers=headers).text 
q2 = BeautifulSoup(site, 'html5lib') 

In [ ]:
from pandas.plotting import table
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Use one of the two methods to get the Senate votes data 
# Filter/Select only those items that are rejected with a margin 
#  of less than 5 votes.  
# Report in a DataFrame 'tableRejVotes' with columns of: 
#  vote number 'Vnum', tally 'Tally', question: description 'Desc', 
#  and issue 'Issue'

df = pd.DataFrame(pd.read_html(q2.prettify(), flavor=['bs4'])[0])
df[['Vote','Tally']] = df['Vote (Tally)'].str.split(expand=True)   
df[['YEAs','NAYs']] = df['Tally'].str.split("-",expand=True)
df['YEAs'] = df.YEAs.str.replace('(', '')
df['NAYs'] = df.NAYs.str.replace(')', '')

df['Tally'] = df['YEAs']+ '-' + df['NAYs']
df['diffvotes'] = abs(df['YEAs'].astype(int)-df['NAYs'].astype(int))

tableRejVotes = df[(df.diffvotes < 5) & (df.Result == 'Rejected')][['Vote','Tally','Question: Description','Issue']]
tableRejVotes.columns=['Vnum','Tally','Desc','Issue']
tableRejVotes['Vnum'] = tableRejVotes['Vnum'].astype('int')
print ("Number of rejected votes with margin < 5 : " + 
        str(tableRejVotes.shape[0]))


Number of rejected votes with margin < 5 : 91


In [ ]:
grader.check("q2")

q2 results: All test cases passed!

## Q3 - (20 pts total) 

From the list of top 200 most popular names for babies born in the US in the 2010s.  We are interested in the names that start with 'W' or contain a letter  'P' or 'p'.  

Create a DataFrame with the names that start with 'W' or contain a letter  'P' or 'p' (for both male and female together), their rank, and the number of times the name is used.  DataFrame has four columns: "Rank" - int, "Names" - str, "Number" - int, and "Gender" - str - "Male"/"Female". 

Examples names from the table are: 
* 4, William, 159773
* 37, Wyatt, 87644
* 118, Willow, 26999
* 44, Penelope, 47570
* 64, Peyton, 39582
* ... 

Display the first 10 rows of the DataFrame, sorted by decreasing number.  Also, print out the sum of the number of the 'W' names and the sum of the number of names with 'P' or 'p'.  

Data available at: https://www.ssa.gov/OACT/babynames/decades/names2010s.html

In [ ]:
site = requests.get('https://www.ssa.gov/OACT/babynames/decades/names2010s.html').text
q3 = BeautifulSoup(site, 'html5lib')

In [ ]:
df = pd.DataFrame(pd.read_html(q3.prettify(), flavor=['bs4'])[0])
dfmales = df['Males']
dfmales = dfmales.drop(labels=200, axis=0)
dfmales['Gender'] = 'Male'
convert_dict = {'Name': object,'Number':np.int64,'Gender': object}
dfmales = dfmales.astype(convert_dict)
dfmales['Rank'] = dfmales['Number'].rank(ascending=False)
dfmales = dfmales.reindex(columns=['Rank','Name','Number','Gender'])
convert_dict = {'Rank': np.int64,'Name': object,'Number':np.int64,'Gender': object}
dfmales = dfmales.astype(convert_dict)
# dfmales

dffemales = df['Females']
dffemales = dffemales.drop(labels=200, axis=0)
dffemales['Gender'] = 'Female'
convert_dict = {'Name': object,'Number':np.int64,'Gender': object}
dffemales = dffemales.astype(convert_dict)
dffemales['Rank'] = dffemales['Number'].rank(ascending=False)
dffemales = dffemales.reindex(columns=['Rank','Name','Number','Gender'])
convert_dict = {'Rank': np.int64,'Name': object,'Number':np.int64,'Gender': object}
dffemales = dffemales.astype(convert_dict)
# dffemales

df2 = pd.concat([dfmales,dffemales])
# df2
# dfW = df2[df2.Name.str.contains('W')]#, regex=False)]
# dfP = df2[df2.Name.str.contains('P')]#, regex=False)]
# dfp = df2[df2.Name.str.contains('p')]#, regex=False)]
dfpatt = df2[(df2.Name.str.contains('W'))|(df2.Name.str.contains('P'))|(df2.Name.str.contains('p'))]
dfpatt = dfpatt.rename(columns={'Number': 'Num'})
dfpatt = dfpatt.sort_values(by=['Num'], ascending=False)
# dfpatt

In [ ]:
from ast import Name
# Create a DataFrame 'names' for the names that start with W or have a P/p
# The 'names' DataFrame has four colummns: 
#   'Rank' (int) 
#   'Name' (str)
#   'Num' (int) 
#   'Gender' (str) 'Female' / 'Male'

names = dfpatt

numWnames = dfpatt[dfpatt.Name.str.contains('W')]['Num'].sum()   # the sum of the names beginning with W 
numPnames = dfpatt[dfpatt.Name.str.contains('P') | dfpatt.Name.str.contains('p')]['Num'].sum()   # the sum of the names with a P/p



print("Total number of names that start with W  : " + str(numWnames))
print("Total number of names that contain P/p   : " + str(numPnames))

names.head(10)

Total number of names that start with W  : 334332
Total number of names that contain P/p   : 895223


,Rank,Name,Num,Gender
2,3,Sophia,180953,Female
3,4,William,159773,Male
19,20,Joseph,115497,Male
26,27,Christopher,101968,Male
36,37,Wyatt,87644,Male
15,16,Harper,85111,Female
82,83,Cooper,49232,Male
83,84,Parker,49187,Male
43,44,Penelope,47570,Female
63,64,Peyton,39582,Female


In [ ]:
grader.check("q3")

q3 results: All test cases passed!

## Q4 - (20 pts total) 

Use Python to scrape the articles posted to https://techcrunch.com/  front page under "The Latest", and report this in a DataFrame.  

The columns of the DataFrame `news` should be:

* 'Title' - article title, type `object` *string*
* 'Date' - publish date, type `datetime64[ns, tz]` _will have to convert to this type_
* 'Author' - author, type `object` *string*

Note: be respectful to the website (obey robots.txt information) and do not repeatedly access the site.


In [ ]:
html = requests.get("https://techcrunch.com").text
soup = BeautifulSoup(html, 'html5lib')

In [ ]:
a1 = soup.find(class_="river river--homepage")
# print(a1.find_all('header'))
articles = []

for i in a1.find_all('header'):
    title = i.find(class_="post-block__title__link").text.strip()
    author = i.find(class_="river-byline__authors").text.strip().replace('\t','')
    datetime = i.find('time')['datetime']
    article = {"title": title, "datetime": datetime, "author": author}
    articles.append(article)
# print(articles) 
q4 = pd.DataFrame.from_dict(articles)
# q4['datetime'] = pd.to_datetime(q4.datetime).dt.tz_localize(None)
q4['datetime'] = q4['datetime'].apply(pd.to_datetime, utc=True)
# q4['datetime'] = pd.Timestamp(q4['datetime'], tz='UTC')
# df['datetime'] = pd.to_datetime(q4['date'], format='%b %d, %Y')
# q4

In [ ]:
# Eastern Daylight Time (EDT) is 4 hours behind Coordinated Universal Time (UTC).

# news.dtypes

title                    object
datetime    datetime64[ns, UTC]
author                   object
dtype: object

In [ ]:
# Pull information from website into DataFrame 'news'
#  'news' has columns: 
#     - 'Title', str 
#     - 'Date', datetime object 
#     - 'Author', str 

news = q4

news

,title,datetime,author
0,"Why Cruise is making its own chips, and a lot ...",2022-10-16 23:05:56+00:00,Roberto Baldwin
1,"Top climate tech deals net nearly $4B in Q3, o...",2022-10-16 18:00:47+00:00,Tim De Chant
2,"With a $13B valuation, Celonis defies current ...",2022-10-16 17:00:14+00:00,Ron Miller
3,Even decacorns have their challenges,2022-10-16 14:16:24+00:00,Mary Ann Azevedo
4,India launches 75 digital banking units across...,2022-10-16 08:32:24+00:00,Manish Singh
5,"Meta announces legs, Hulu raises prices, and M...",2022-10-15 20:15:36+00:00,Greg Kumparak
6,Zombie startups,2022-10-15 18:01:54+00:00,Natasha Mascarenhas
7,"This Week in Apps: Play Store revamp, Google a...",2022-10-15 18:00:25+00:00,Sarah Perez
8,"Pay as you drive, or pay how you drive?",2022-10-15 17:00:32+00:00,Anna Heim
9,"Starlink isn’t a charity, but the Ukraine war ...",2022-10-15 15:02:59+00:00,Devin Coldewey


In [ ]:
grader.check("q4")

q4 results: All test cases passed!

<!-- BEGIN QUESTION -->

## Bonus - (4 points) 

Going back to the names data from Q3.  Let's look at how the distribution of popular names has changed over time. 

Create a small multiples plot (meaning each plot is the same type and showing similar information in a grid).  

The small multiples plot will show how the distribution of names has changed from the 1960s to 2010s.  

1960s data - https://www.ssa.gov/OACT/babynames/decades/names1960s.html  
1970s data - https://www.ssa.gov/OACT/babynames/decades/names1970s.html  
...  
2010s data - https://www.ssa.gov/OACT/babynames/decades/names2010s.html

Let's focus on just how the distribution of girls names have changed.  This can be displayed using a line plot on just the top 100 names per decade.

When getting the information, you should also scrape the total number of female births in that period to normalize the number of names as a percent in that decade. 


Your final grid should display: 

|         | Col 1 | Col 2 | 
|-------|-----------|-----------|
| Row 1 | 1960 plot | 1970 plot | 
| Row 2 | 1980 plot | 1990 plot | 
| Row 3 | 2000 plot | 2010 plot | 



In [ ]:
# Create small multiples plot. 

...


<!-- END QUESTION -->

